<a href="https://colab.research.google.com/github/shengjinbiao/party/blob/main/Party_%E6%A8%A1%E5%9E%8B%E5%8E%9F%E7%90%86%E8%88%87%E4%B8%AD%E6%96%87%E8%99%95%E7%90%86%E8%A7%A3%E6%9E%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧠 Party 模型原理與中文處理解析

## 一、什麼是 Party？

Party（PAge-wise Recognition of Text-y）是一種新型的文字識別模型，專為自動文字識別（ATR）系統設計。它的核心理念是：**按頁識別文字**，而非傳統的逐行或逐字識別。這使得 Party 特別適合處理排版複雜、語言多樣的歷史文獻與手稿。

Party 的設計目標是取代傳統依賴「基線 + 邊界多邊形」或「文字框」的 OCR 模型。它透過更簡化的提示方式（如基線或框）來理解文字行的位置，**不再需要繁瑣的多邊形標註**。

---

## 二、模型架構解析

Party 模型由三個主要模塊組成：

- 🎯 **Swin Vision Transformer 編碼器**  
  用於提取整頁圖像的視覺特徵，具備強大的空間理解能力。

- 📐 **Baseline Positional Embeddings**  
  將每行文字的基線位置嵌入模型，作為空間提示，幫助模型定位文字。

- 🦙 **Tiny Llama 解碼器（Octet Tokenization）**  
  使用小型 Llama 模型進行文字生成，並採用 UTF-8 的八位元編碼方式（octet tokenization）來預測文字。

---

## 三、Octet Tokenization：如何處理中文？

中文字符的 Unicode 編碼通常超過 8 位元，例如「謝」是 `U+8B1D`。Party 並不直接預測 Unicode 字元，而是將其轉換為 UTF-8 編碼後，**逐 byte 預測**：

### 範例：「謝」

| 編碼步驟 | 二進位 | 十六進位 |
|----------|--------|----------|
| Byte 1   | 11101000 | E8       |
| Byte 2   | 10101100 | AC       |
| Byte 3   | 10011101 | 9D       |

Party 模型會依序生成 `E8 → AC → 9D`，最後再由後處理程序還原成「謝」。這種方式的優點是：

- ✅ 支援任意語言與符號（只要能轉成 UTF-8）
- ✅ 不需事先定義字元集
- ✅ 模型架構更簡潔、語言無關

但也有挑戰：

- ❗ 若 byte 預測錯誤，可能導致無法還原成合法字元
- ❗ 輸出可能不符合 Unicode 正規化（如組合音符）

---

## 四、語言標記與中文推理

Party 的最新版本引入了**語言標記（language tokens）**，可在推理時指定每行的語言，避免模型在多語言文獻中隨機切換語言。對於中文文獻，建議明確指定語言標記（如 `-l zho`），以提升準確率。zho是ISO 639-2和639-3標準表示中文的的語言代碼，繁體通常用zh-Hant或者cmn-Hant，簡體用zh-Hans，或cmn-Hans，這些是ISO-15924中的腳本代碼。

若未指定語言，模型會自動生成語言標記，判斷該行可能包含的語言。編譯訓練數據集的時候，將訓練數據放在chinese目錄下面，如果是繁體中文放在classical_chinese，簡體用simplified_chinese之類的，dataset/classical_chinese/data_1/*.xml，這樣模型在訓練的時候能自動讀取語言信息。如果沒有任何指定，在中文古籍中，這可能導致誤判，尤其是混合使用文言與白話的情況。

---

## 五、微調建議與中文應用

雖然 Party 在訓練資料中常見語言上表現良好，但對於中文等非主流語言，**仍建議進行微調**，以符合具體的轉寫準則與排版特性。

微調流程包括：

1. 使用 ALTO 或 PageXML 格式的標註資料（含基線與文字）
2. 編譯成 Party 專用格式：  
   `party compile -o dataset.arrow *.xml`
3. 訓練模型：  
   `party train --load-from-repo ... --prompt-mode curves`

---

## 六、總結

Party 模型以其創新的頁面級識別方式、靈活的 octet 編碼策略與語言標記機制，為多語言文獻處理提供了強大的工具。對於中文文獻，尤其是古籍與手稿，Party 的架構提供了極具潛力的識別能力，只需適度微調即可達到實用水準。

如果你正在進行中文 OCR 項目，Party 值得深入探索與應用。


In [ ]:
#加载谷歌硬盘，用于永久保存文件
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#安装party, 更改运行时需要重新安装

!pip install git+https://github.com/mittagessen/party.git --retries 5 --timeout 120

  Cloning https://github.com/mittagessen/party.git to /tmp/pip-req-build-9cxhfc0v
  Running command git clone --filter=blob:none --quiet https://github.com/mittagessen/party.git /tmp/pip-req-build-9cxhfc0v
  Resolved https://github.com/mittagessen/party.git to commit 271d6a7cc2720d068004ac1e3e5d19886ff14cc5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/mittagessen/kraken.git (to revision 348ff35) to /tmp/pip-install-w6jkei87/kraken_aac615a243404a57ae27399f13eccd89
  Running command git clone --filter=blob:none --quiet https://github.com/mittagessen/kraken.git /tmp/pip-install-w6jkei87/kraken_aac615a243404a57ae27399f13eccd89
  Running command git checkout -q 348ff35
  Resolved https://github.com/mittagessen/kraken.git to commit 348ff35
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  

In [ ]:
#转换模型格式，便于在不同平台使用，或者测试模型效果如下
!party convert -o /content/drive/MyDrive/party/model/model_stageE/model_stageE.safetensors /content/drive/MyDrive/party/model/model_stageE/checkpoint_05-0.0500.ckpt

Output file written to /content/drive/MyDrive/party/model/model_stageE/model_stageE.safetensors


In [ ]:
# 测试模型的效果
!party -d cuda:0 test --curves --load-from-file /content/drive/MyDrive/party/model/model_stageE/model_stageE.safetensors  /content/drive/MyDrive/party/dataset_val.arrow

Compiling model ✓
Files             ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% 0/37    -:--:-- 0:00:00
Files             ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% 0/37 -:--:-- 0:00:01
Files             ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% 0/37 -:--:-- 0:00:02
Files             ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% 0/37 -:--:-- 0:00:03
Files             ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% 0/37 -:--:-- 0:00:04
Files             ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% 0/37 -:--:-- 0:00:05
Files             ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% 0/37 -:--:-- 0:00:06
Files             ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% 0/37 -:--:-- 0:00:07
Files             ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   0% 0/37 -:--:-- 0:00:08
Files             ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   3% 1/37    -:--:-- 0:00:08
Files             ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   3% 1/37 -:--:-- 0:00:09
Files             ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━   3% 1/37 -:--:-- 0:00:10
Files     

# Task
Check if the training and validation `.arrow` datasets and their corresponding `.lst` files have been successfully created and are ready for model training. Specifically, verify the existence of the files: `/content/drive/MyDrive/party/dataset_train.arrow`, `/content/drive/MyDrive/party/dataset_val.arrow`, `/content/drive/MyDrive/party/train.lst`, and `/content/drive/MyDrive/party/val.lst`.

# 編譯數據集

兼數據目錄複製到谷歌雲盤，ALTo格式的文件包解壓，包含圖片和xml，然腳本發現目錄下面所有xml文件，按照9:1或者8:2隨機分割數據，然後編譯成.arrow格式的訓練集和驗證集。

## Discover XML Files

### Subtask:
递归查找 `/content/drive/MyDrive/party/datasets/classical_chinese` 目录下所有的 `.xml` 文件。


**Reasoning**:
To find all XML files recursively, I will use the `find` command within a code cell and capture its output to a Python list.



In [ ]:
import subprocess

# Define the directory to search
directory_to_search = '/content/drive/MyDrive/party/datasets/traditional_chinese'

# Use the find command to locate all .xml files recursively
find_command = ['find', directory_to_search, '-name', '*.xml']

try:
    # Execute the command and capture stdout
    result = subprocess.run(find_command, capture_output=True, text=True, check=True)
    xml_files = result.stdout.strip().split('\n')
    # Filter out any empty strings that might result from split()
    xml_files = [f for f in xml_files if f]

    print(f"Found {len(xml_files)} XML files.")
    # Optionally, print the first few files to verify
    if xml_files:
        print("First 5 XML files found:")
        for f in xml_files[:5]:
            print(f)
    else:
        print("No XML files found in the specified directory.")

except subprocess.CalledProcessError as e:
    print(f"Error executing find command: {e.stderr}")
    xml_files = []
except FileNotFoundError:
    print(f"Error: The directory '{directory_to_search}' does not exist or 'find' command is not found.")
    xml_files = []

Found 373 XML files.
First 5 XML files found:
/content/drive/MyDrive/party/datasets/classical_chinese/vol-16/02086213.cn.pdf_page_1.xml
/content/drive/MyDrive/party/datasets/classical_chinese/vol-16/02086213.cn.pdf_page_4.xml
/content/drive/MyDrive/party/datasets/classical_chinese/vol-16/02086213.cn.pdf_page_2.xml
/content/drive/MyDrive/party/datasets/classical_chinese/vol-16/02086213.cn.pdf_page_3.xml
/content/drive/MyDrive/party/datasets/classical_chinese/vol-16/02086213.cn.pdf_page_36.xml


## Split Data

### Subtask:
将发现的XML文件随机分割为训练集（90%）和验证集（10%）。


**Reasoning**:
To split the XML files into training and validation sets, I need to first shuffle the list and then determine the split point based on the requested percentages.



In [ ]:
import random

# Shuffle the xml_files list randomly
random.shuffle(xml_files)

# Calculate the split point (90% for training, 10% for validation)
split_point = int(len(xml_files) * 0.9)

# Split the files into training and validation sets
train_files = xml_files[:split_point]
val_files = xml_files[split_point:]

print(f"Total XML files: {len(xml_files)}")
print(f"Training files: {len(train_files)} (90%)")
print(f"Validation files: {len(val_files)} (10%)")

Total XML files: 373
Training files: 335 (90%)
Validation files: 38 (10%)


**Reasoning**:
Now that the XML files are split into training and validation sets, I will write these file paths to respective '.lst' files, which are often used by machine learning tools like 'party' to specify data lists.



In [ ]:
import os

# Define output directories for the .lst files
output_dir = '/content/drive/MyDrive/party/'

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Write training file paths to train.lst
train_lst_path = os.path.join(output_dir, 'train.lst')
with open(train_lst_path, 'w') as f:
    for file_path in train_files:
        f.write(file_path + '\n')
print(f"Training file list written to: {train_lst_path}")

# Write validation file paths to val.lst
val_lst_path = os.path.join(output_dir, 'val.lst')
with open(val_lst_path, 'w') as f:
    for file_path in val_files:
        f.write(file_path + '\n')
print(f"Validation file list written to: {val_lst_path}")


Training file list written to: /content/drive/MyDrive/party/train.lst
Validation file list written to: /content/drive/MyDrive/party/val.lst


**Reasoning**:
The previous steps successfully created the `.lst` files. Now, I need to verify that both the generated `.lst` files and the `.arrow` datasets exist in the specified paths before proceeding, as per the main task's instruction to check if they are ready for model training.



In [ ]:
import os

def check_file_existence(file_path):
    if os.path.exists(file_path):
        print(f"SUCCESS: File '{file_path}' exists.")
        return True
    else:
        print(f"FAILURE: File '{file_path}' does NOT exist.")
        return False

# Paths to check
train_arrow_path = '/content/drive/MyDrive/party/dataset_train.arrow'
val_arrow_path = '/content/drive/MyDrive/party/dataset_val.arrow'
train_lst_path = '/content/drive/MyDrive/party/train.lst'
val_lst_path = '/content/drive/MyDrive/party/val.lst'

all_files_exist = True
all_files_exist &= check_file_existence(train_arrow_path)
all_files_exist &= check_file_existence(val_arrow_path)
all_files_exist &= check_file_existence(train_lst_path)
all_files_exist &= check_file_existence(val_lst_path)

if all_files_exist:
    print("All required training and validation files are present.")
else:
    print("One or more required training or validation files are missing. Please review previous steps.")


SUCCESS: File '/content/drive/MyDrive/party/dataset_train.arrow' exists.
SUCCESS: File '/content/drive/MyDrive/party/dataset_val.arrow' exists.
SUCCESS: File '/content/drive/MyDrive/party/train.lst' exists.
SUCCESS: File '/content/drive/MyDrive/party/val.lst' exists.
All required training and validation files are present.


## Compile Training Dataset

### Subtask:
使用训练集中的XML文件编译生成Party专用的 `.arrow` 格式数据集，并保存为 `/content/drive/MyDrive/party/dataset_train.arrow`。


**Reasoning**:
To compile the training dataset, I will read the list of training XML files from `train.lst` and pass them as arguments to the `party compile` command, along with the specified output path for the `.arrow` file.



In [ ]:
import os

# Define paths
train_lst_path = '/content/drive/MyDrive/party/train.lst'
output_arrow_path = '/content/drive/MyDrive/party/dataset_train.arrow'

# Read the list of training XML files
with open(train_lst_path, 'r') as f:
    train_xml_files = [line.strip() for line in f if line.strip()]

# Construct the party compile command
# The command line interface for `party compile` expects input files directly as arguments.
compile_command = ['party', 'compile', '-o', output_arrow_path] + train_xml_files

# Execute the command
print(f"Executing command: {' '.join(compile_command)}")
!{' '.join(compile_command)}

print(f"Training dataset compiled to: {output_arrow_path}")

Executing command: party compile -o /content/drive/MyDrive/party/dataset_train.arrow /content/drive/MyDrive/party/datasets/classical_chinese/vol-1718/十七卷4-5-1.pdf_page_1.xml /content/drive/MyDrive/party/datasets/classical_chinese/vol-1718/02086214.cn.pdf_page_140.xml /content/drive/MyDrive/party/datasets/classical_chinese/vol-1718/02086214.cn.pdf_page_58.xml /content/drive/MyDrive/party/datasets/classical_chinese/vol-16/02086213.cn.pdf_page_54.xml /content/drive/MyDrive/party/datasets/classical_chinese/vol-15/02086212.cn.pdf_page_43.xml /content/drive/MyDrive/party/datasets/classical_chinese/vol-16/02086213.cn.pdf_page_31.xml /content/drive/MyDrive/party/datasets/classical_chinese/vol-1718/02086214.cn.pdf_page_11.xml /content/drive/MyDrive/party/datasets/classical_chinese/vol-16/02086213.cn.pdf_page_16.xml /content/drive/MyDrive/party/datasets/classical_chinese/vol-15/02086212.cn.pdf_page_53.xml /content/drive/MyDrive/party/datasets/classical_chinese/vol-1718/02086214.cn.pdf_page_114.x

# Task
**Compile Validation Dataset**: 使用验证集中的XML文件编译生成Party专用的 `.arrow` 格式数据集，并保存为 `/content/drive/MyDrive/party/dataset_val.arrow`。

## Compile Validation Dataset

### Subtask:
使用验证集中的XML文件编译生成Party专用的 `.arrow` 格式数据集，并保存为 `/content/drive/MyDrive/party/dataset_val.arrow`。


**Reasoning**:
To compile the validation dataset, I will read the list of validation XML files from `val.lst` and pass them as arguments to the `party compile` command, along with the specified output path for the `.arrow` file.

